In [1]:
import snap
import time, gensim, pickle, json, networkx as nx, numpy as np
from networkx.readwrite import json_graph

In [2]:
with open('../REST/static/networks/twitter_users_graph2.json', 'r') as f:
    data = json.load(f)
    nxg = json_graph.node_link_graph(data, directed=True)
nxg = nxg.to_directed()

In [3]:
g = snap.PUNGraph.New()

In [4]:
for n in nxg.nodes():
    g.AddNode(n)

In [5]:
for f,t in nxg.edges():
    g.AddEdge(f, t)

In [6]:
CmtyV = snap.TCnComV()

In [7]:
start = time.time()
modularity = snap.CommunityGirvanNewman(g, CmtyV)
print(time.time()-start)

5.47504687309


In [8]:
nodes_communities = {} # {node: [community]}
for i, Cmty in enumerate(CmtyV):
    print ("Community: ")
    for NI in Cmty:
        nodes_communities.setdefault(NI, [])
        nodes_communities[NI].append(i)
        print (NI)
print ("The modularity of the network is %f" % modularity)

Community: 
39937481
66645671
97633683
106374774
134531097
162135761
165047989
234282464
310141719
314762385
315757507
319391792
334697739
339384203
350791386
459012541
734074579
762683270
786513674
946797248
965098470
979769992
995806488
Community: 
105809133
140011527
141529642
154451555
157053610
159967656
179954793
181276229
205640168
234536504
266826023
271721714
276349436
356478549
469654164
518787696
550121465
715580484
841745956
847176660
925109031
939057318
Community: 
94549014
111450444
139032492
147170681
147572001
154069721
162427198
201203999
218922545
304139482
321885193
322601600
372916058
492408930
819172968
826480844
891177368
939422778
Community: 
55220437
74959905
121806922
121914206
141306910
166720591
197219379
218862394
236914275
281985346
336584201
792520470
803581588
822909215
837023808
944199207
964125915
Community: 
113712545
149522762
166509333
218228766
228970729
246040835
265254213
287435569
309416262
331537003
372162808
434723411
476086009
704764994
804034

In [9]:
for node in nxg.nodes():
    nxg.nodes[node]["community"] = " ".join([str(com) for com in nodes_communities[node]])

In [10]:
for ix,deg in nxg.degree(nxg.nodes()):
    nxg.node[ix]['degree'] = deg

for ix,in_deg in nxg.in_degree(nxg.nodes()):
    nxg.node[ix]['in_degree'] = in_deg
    
for ix,out_deg in nxg.out_degree(nxg.nodes()):
    nxg.node[ix]['out_degree'] = out_deg

pagerank = nx.pagerank(nxg)

for ix,v in pagerank.items():
    nxg.node[ix]["pagerank"] = v

In [11]:
def networkx_to_snappy(nxg, directed=False):
    if directed:
        g = snap.PNGraph.New()
    else:
        g = snap.PUNGraph.New()
        
    for n in nxg.nodes():
        g.AddNode(n)
    for f,t in nxg.edges():
        g.AddEdge(f, t)
        
    return g

In [12]:
data = nx.node_link_data(nxg)
data["modularity"] = modularity
data["number_of_communities"] = len(CmtyV)

snappy_directed = networkx_to_snappy(nxg, True)
snappy_undirected = networkx_to_snappy(nxg, True)

diam_directed = snap.GetBfsFullDiam(snappy_directed, 100, True)
diam_undirected = snap.GetBfsFullDiam(snappy_undirected, 100, False)

data["diameter_directed"]=diam_directed
data["diameter_undirected"]=diam_undirected
with open('../visualization/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)
    
with open('../REST/static/networks/twitter_users_graph2.json', 'w') as f:
    json.dump(data, f, indent=4)

In [13]:
json_graph.node_link_data(json_graph.node_link_graph(data, directed=True)).keys()

['directed', 'graph', 'nodes', 'links', 'multigraph']